In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/my_data_DS317/train_update.xlsx')
df.drop(['gioitinh'], axis=1)
df.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,39B39397XPvAibaEXe/vGZ1xWLvpK7wMZlgBlCBs,19,0,75.198135,30.0,9.37,9.37,24,9.37,9.34,9.40,9.370000,9.370000,9.370000,9.370000,9.370000
1,3AB392AFXPvAibaEXe/NRwqjPAC0MKjf0QSiSLjM,17,0,85.000000,26.5,7.07,7.07,139,7.00,6.74,6.25,6.230000,7.150000,6.920000,6.970000,7.600000
2,BEC54CF0XPvAibaEXe/qZKhoIW7F1rbPON7lHJUG,19,0,82.000000,30.0,6.54,6.54,18,6.54,6.54,6.54,6.540000,6.540000,6.540000,6.540000,6.540000
3,43386D57XPvAibaEXe/nw3DhNQnHfzJZvI0o00SJ,20,0,75.655535,22.0,1.83,1.83,4,3.56,0.00,1.78,1.780000,1.780000,1.780000,1.780000,1.780000
4,8F1A725FXPvAibaEXe88DMpF1KL2K3HyZfgFjAif,19,0,100.000000,30.0,9.31,9.31,64,9.21,9.32,9.41,9.313333,9.313333,9.313333,9.313333,9.313333


In [ ]:
df_test = pd.read_excel('/content/drive/MyDrive/my_data_DS317/test_update.xlsx')
df_test.drop(['gioitinh'], axis=1)
df_test.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,14EEA146XPvAibaEXe/2sgPA+OKAks3AV3zsZSFH,22,0,82.105913,26.762789,8.77,8.77,123,8.61,8.79,9.01,9.04,8.63,8.620,8.530,8.747143
1,4A2628C7XPvAibaEXe9urngX21O5XaWExeziRFls,22,1,82.105913,30.000000,8.42,8.42,130,9.11,8.64,7.78,8.60,8.45,8.300,9.300,8.597143
2,44634117XPvAibaEXe+WdrMfsV/1fSPAGTu3hQ5U,22,0,82.105913,22.350000,7.73,7.73,137,7.83,7.46,7.53,7.67,7.89,8.030,7.750,7.737143
3,8A7BA5ACXPvAibaEXe8FNnc+c+WuJHdW9KyIp7Nl,19,0,100.000000,30.000000,6.52,6.52,83,7.63,7.21,7.70,7.05,1.67,6.252,6.252,6.252000
4,A3A77666XPvAibaEXe/4WyTnp6RTwi1hkjCNtuCX,20,0,75.655535,24.250000,6.50,6.50,148,6.02,6.75,6.79,6.26,5.21,5.580,6.450,6.690000


In [ ]:
# Danh sách các mô hình
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "KNN": KNeighborsRegressor(),
    "MLP": MLPRegressor(random_state=42, max_iter=1000),
    "XGBoost": XGBRegressor(random_state=42)
}

# Độ đo
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

# Cột ban đầu cho X
initial_columns = ['namsinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns)+2, len(df.columns) - 1):  # Bắt đầu từ cột tiếp theo sau `initial_columns`
    if i +2 >= len(df.columns):
        break
    target_column = df.columns[i+2]  # Cột y hiện tại
    print(target_column)

    # Xác định X và y
    X_train = df[current_X_columns]
    y_train = df[target_column]

    X_test = df_test[current_X_columns]
    y_test = df_test[target_column]

    # Huấn luyện và đánh giá các mô hình
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Tính toán các độ đo
        metrics = calculate_metrics(y_test, y_pred)
        metrics["Model"] = model_name
        metrics["Target Column"] = target_column
        results.append(metrics)

    # # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)
results_df.to_csv("model_results.csv", index=False)

diemtbhk_2
diemtbhk_3
diemtbhk_4
diemtbhk_5
diemtbhk_6
diemtbhk_7
diemtbhk_8


In [ ]:
results_df

,R2 Score,MSE,RMSE,MAE,MAPE,MPE,Model,Target Column
0,0.789540,0.940306,0.969694,0.628670,NaN,NaN,Linear Regression,diemtbhk_2
1,0.673618,1.458233,1.207573,0.757722,NaN,NaN,Decision Tree,diemtbhk_2
2,0.806810,0.863150,0.929058,0.584717,NaN,NaN,KNN,diemtbhk_2
3,0.810850,0.845098,0.919292,0.606047,NaN,NaN,MLP,diemtbhk_2
4,0.806411,0.864932,0.930017,0.567564,NaN,NaN,XGBoost,diemtbhk_2
5,0.847525,0.605394,0.778071,0.479222,NaN,NaN,Linear Regression,diemtbhk_3
6,0.753912,0.977076,0.988472,0.582819,NaN,NaN,Decision Tree,diemtbhk_3
7,0.849775,0.596459,0.772307,0.491644,NaN,NaN,KNN,diemtbhk_3
8,0.866581,0.529733,0.727828,0.470419,NaN,NaN,MLP,diemtbhk_3
9,0.870711,0.513336,0.716474,0.436151,NaN,NaN,XGBoost,diemtbhk_3


In [ ]:
# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)

# Tính trung bình của các độ đo cho từng model, chỉ tính toán trên các cột số
numeric_columns = results_df.select_dtypes(include=np.number).columns
average_metrics = results_df.groupby("Model")[numeric_columns].mean().reset_index()

# Lưu kết quả ra file CSV
results_df.to_csv("model_results.csv", index=False)
average_metrics.to_csv("average_model_metrics.csv", index=False)

# Hiển thị kết quả trung bình
print("Average metrics for each model:")
print(average_metrics)

Average metrics for each model:
               Model  R2 Score       MSE      RMSE       MAE  MAPE  MPE
0      Decision Tree  0.648357  1.577621  1.228222  0.627636   NaN  NaN
1                KNN  0.792767  0.925068  0.945665  0.516429   NaN  NaN
2  Linear Regression  0.798342  0.899476  0.934621  0.515613   NaN  NaN
3                MLP  0.811902  0.839151  0.900842  0.508529   NaN  NaN
4            XGBoost  0.805754  0.868542  0.912880  0.487342   NaN  NaN
